In [ ]:
# MIT License
#
#@title Copyright (c) 2021 CCAI Community Authors { display-mode: "form" }
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

## 初めに
ASE×NNPの威力を体験していただくため、このnotebookは以下の処理を行います。
1. Materials Projectから既知の結晶構造を入手する
2. 入手した結晶構造に対して、任意表面を切り出す
3. 所望の組成比となるように調整し、表面構造のエネルギーを測定する
4. 上記の表面における分子（例としてCH4）の吸着エネルギーを計算する

当notebookは以下の公知情報を参考に作成されたものです。
https://docs.matlantis.com/atomistic-simulation-tutorial/ja/index.html

## 0. 学習済みモデル（力場）のダウンロード

学習済みのモデルを右記リンクからダウンロードします。OC20+OC22データセット、すなわち金属および金属酸化物に対応した力場を使用します。[here](https://github.com/Open-Catalyst-Project/ocp/blob/master/MODELS.md)

今回はリンク先から該当ファイルを直接ダウンロードし、計算環境のフォルダに格納しました。また、計算環境はGPUを前提としました（OCPCalculatorの引数におけるcpu = False）

In [ ]:
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import ase.io
from ase.optimize import BFGS, LBFGS, FIRE
from ase.build import add_adsorbate, molecule, bulk
import os
from ase.constraints import FixAtoms
import numpy as np
import pandas as pd
import random
from ase.build import molecule, bulk
from ase.build import bcc100, bcc110, bcc111, fcc111, fcc100, fcc110, fcc211, hcp0001, hcp10m10, diamond100, diamond111
from ase.visualize import view
from ase.data import atomic_numbers
import collections
from ase.io.trajectory import Trajectory
from ase.io import extxyz
from pymatgen.io.cif import CifWriter
import ase
from ase import Atoms, units
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view
from ase.build.rotate import minimize_rotation_and_translation
from ase.optimize import BFGS, LBFGS, FIRE
from ase.md import MDLogger
from ase.io import read, write, Trajectory
from ase.build import sort
from ase.visualize import view
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt
from pymatgen.ext.matproj import MPRester
import plotly.express as px


#モデルの置き場所（何かうまくいかなかったので絶対パスで指定）
checkpoint_path = "C:/Users/uni21/ocp/gnoc_oc22_oc20_all_s2ef.pt"
config_yml_path = r"C:\Users\uni21\ocp\configs\oc22\s2ef\gemnet-oc\gemnet_oc_oc20_oc22.yml"

#Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path, cpu = False)

In [ ]:
df_catalyst_key = pd.DataFrame(pd.read_csv("20221229_ocm_catalyst_mp_key.csv"))

In [ ]:
df_catalyst_key = df_catalyst_key.loc[df_catalyst_key['Support '] == 'SiO2']
df_catalyst_key_unique = df_catalyst_key.loc[df_catalyst_key['M2'] == 'Na']

In [ ]:
df_catalyst_key_unique

In [ ]:
def make_crystal_cif(ids):
    MY_API_KEY="WBBfnnIT4C97428wTRakQvRyXNBq2pj5"

    #from pymatgen import MPRester
    with MPRester(MY_API_KEY) as mpr:
        docs = mpr.materials.search(material_ids=["mp-" + str(ids)])
        #入手した結晶構造の確認
        example_doc = docs[0]
        structure = example_doc.structure
        #結晶構造をcifファイルに格納
        w = CifWriter(structure)
        w.write_file('mystructure.cif')

In [ ]:
def read_cif_and_set_positions():
    bulk = read("mystructure.cif")
#    print("Number of atoms =", len(bulk))
#    print("Initial lattice constant =", bulk.cell.cellpar())
    tags = np.ones(len(bulk))
    bulk.set_tags(tags)
    #繰り返し構造の設定
    bulk = bulk.repeat([2, 2, 2])
    bulk = sort(bulk)
    # Shift positions a bit to prevent surface to be cut at wrong place when `makesurface` is called
    bulk.positions += [0.01, 0, 0]
    return bulk

In [ ]:
bulk_test = read_cif_and_set_positions()
#view(bulk_test)

In [ ]:
fcc_energies = []
for i in np.linspace(0.9, 1.1):
    bulk_test = read_cif_and_set_positions()
#    bulk_test = modify_compositions(insert_num, base_num, insert_ratio, base_ratio, bulk_test)
    ref = bulk_test.cell
    bulk_test.cell = i * ref
    print(bulk_test.cell)
    bulk_test.set_tags(np.ones(len(bulk_test)))

    bulk_test.calc = calc

    e = (bulk_test.get_potential_energy())
    fcc_energies.append(e)
    del bulk_test, ref

    


In [ ]:
plt.plot(np.linspace(0.9, 1.1), fcc_energies)

In [ ]:
def bulk_set_opt(insert_num, base_num, insert_ratio, base_ratio):
    fcc_energies = []
    for i in np.linspace(0.9, 1.1):
        bulk_test = read_cif_and_set_positions()
        bulk_test = modify_compositions(insert_num, base_num, insert_ratio, base_ratio, bulk_test)
        ref = bulk_test.cell
        bulk_test.cell = i * ref
        print(bulk_test.cell)
        bulk_test.set_tags(np.ones(len(bulk_test)))

        bulk_test.calc = calc

        e = (bulk_test.get_potential_energy())
        fcc_energies.append(e)
        del bulk_test, ref
        
    bulk_energy = pd.DataFrame([fcc_energies, np.linspace(0.9, 1.1)])
    opt_i = get_column_with_min_value_in_row_zero(bulk_energy)[1]

    bulk_test = read_cif_and_set_positions()
    bulk_test = modify_compositions(insert_num, base_num, insert_ratio, base_ratio, bulk_test)
    ref = bulk_test.cell
    bulk_test.cell = opt_i * ref
    print(bulk_test.cell)
    return bulk_test


In [ ]:
#bulk_energy = pd.DataFrame([fcc_energies, np.linspace(0.9, 1.1)])
#min(bulk_energy.iloc[0,:])
#plt.plot(np.linspace(0.9, 1.1), fcc_energies)

In [ ]:
def get_column_with_min_value_in_row_zero(df):
    # 0行目の値が最小となる列のインデックスを取得
    min_col_index = df.iloc[0].idxmin()

    # 対象の列を抽出
    min_col = df[min_col_index]

    return min_col

In [ ]:
#get_column_with_min_value_in_row_zero(bulk_energy)[1]

In [ ]:
def makesurface(
    atoms: Atoms, miller_indices=(1, 1, 1), layers=3, rep=[4, 4, 1]
) -> Atoms:
    s1 = surface(atoms, miller_indices, layers)
    s1.center(vacuum=10.0, axis=2)
    s1 = s1.repeat(rep)
    s1.set_positions(s1.get_positions() - [0, 0, min(s1.get_positions()[:, 2])])
    s1.pbc = True
    return s1

In [ ]:
def modify_compositions(insert_num, base_num, insert_ratio, base_ratio, slab):
    #結晶構造の元素番号を確認する
    numbers = slab.get_atomic_numbers()
    #base_atomの個数
    base_count = np.count_nonzero(numbers == base_num)
    #base_atomのindex
    indices_base_count = np.where(numbers == base_num)[0]
    #base_atomのindexを、任意比率でinsert_atomに変更
    np.random.seed(1)
    replace_index = np.random.choice(indices_base_count, size= round(base_count*insert_ratio / (insert_ratio + base_ratio)), replace=False)
    print(f"Replace {replace_index} atom")
    numbers[replace_index] = insert_num
    slab.set_atomic_numbers(numbers)
    return slab

In [ ]:
adsorption_structures = pd.DataFrame(np.empty((1, 6)), columns = ["composition", "opt_energy", "mol", "onto_0", "onto_1", "ads_energy"])

In [ ]:
df_catalyst_key_unique["Name"][df_catalyst_key_unique.index[1]].replace("/","_")

In [ ]:
methane = molecule("CH4")
methane.rotate(30, v = "x")
methane.rotate(30, v = "y")
#view(methane)

In [ ]:
methane.positions

In [ ]:
methanol = molecule("CH3OH")
methanol.rotate(90, v = "x")
methanol.positions[1] = [0, 0, -1.5]
methanol.positions[3] = [0, 0, -0.47]
#view(methanol)

#methanol.rotate(60, v = "y")
#O = methanol.positions[1]
#H = methanol.positions[3]
#print(O)
#print(H)

#methanol_rev = methanol
#methanol_rev.positions[1] = [-4.68724263e-01, -6.41995367e-17, -1.28506176e+00]
#methanol_rev.positions[3] = [-6.80489936e-01, -4.64478527e-17, -2.38458857e-01]
#view(methanol_rev)

In [ ]:
oxygen = molecule("O")
methoxy = molecule("CH3O")
methan_ol = molecule("CH3OH")

In [ ]:
molec = [methane, methanol, oxygen, methoxy]
molec

In [ ]:
for index in range(len(df_catalyst_key_unique)):
    ids = df_catalyst_key_unique["Material ID"][df_catalyst_key_unique.index[index]]
    insert_num = df_catalyst_key_unique["M1_atom_number"][df_catalyst_key_unique.index[index]]
    base_num = df_catalyst_key_unique["M2_atom_number"][df_catalyst_key_unique.index[index]]
    insert_ratio = df_catalyst_key_unique["M1_mol%"][df_catalyst_key_unique.index[index]]
    base_ratio = df_catalyst_key_unique["M2_mol%"][df_catalyst_key_unique.index[index]]
    name = df_catalyst_key_unique["Name"][df_catalyst_key_unique.index[index]].replace("/","_")
    
    
    make_crystal_cif(ids)
    bulk = bulk_set_opt(insert_num, base_num, insert_ratio, base_ratio)
    
    slab = makesurface(bulk, miller_indices=(1, 1, 1), layers=2, rep=[1, 1, 1])
    slab = sort(slab)
    # adjust `positions` before `wrap`
    slab.positions += [1, 1, 0]
    slab.wrap()
    
    # Check z_position of atoms
    atoms = slab
    df = pd.DataFrame({
        "x": atoms.positions[:, 0],
        "y": atoms.positions[:, 1],
        "z": atoms.positions[:, 2],
        "symbol": atoms.symbols,
    })
    #display(df)

    coord = "z"
    df_sorted = df.sort_values(coord).reset_index().rename({"index": "atom_index"}, axis=1)
    z_threshold = df_sorted["z"][len(df_sorted) - 4]
#    z_threshold = 20
    
#    slab = modify_compositions(insert_num, base_num, insert_ratio, base_ratio, slab)
    
    # Fix atoms under z=10 A（subsurfaceの原子は固定、この束縛は任意に緩和可能だが緩和するほど計算は重くなる）
    c = FixAtoms(indices=[atom.index for atom in slab if atom.position[2] <= z_threshold])
    slab.set_constraint(c)
    slab.center(vacuum=10.0, axis=2)
    slab.set_pbc(True)
    slab.set_calculator(calc)
    
    os.makedirs("output", exist_ok=True)
    BFGS_opt = LBFGS(slab, trajectory="output/slab_opt" + name + ".traj")#, logfile=None)
    BFGS_opt.run(fmax=0.05, steps = 200)
    
    # Save slab structure
    os.makedirs("output/structures/", exist_ok=True)
    write("output/structures/Slab_" + name + ".xyz", slab)
    
    #スラブモデルのエネルギーを保存
    slab_energy = slab.get_potential_energy()
    #各吸着点における候補分子の吸着構造最適化とエネルギー計算
    for i in range(1, 4, 1):
        for j in range(1, 3, 1):
            for mol in molec:
#                mol = molecule(mol)
#                mol.rotate(30, v="x")
#                mol.rotate(30, v="y")
                mol_on_slab = slab.copy()

                # height: height of molecule from slab surface
                # position: x,y position of molecule
                # The molecule position can be modified later, and thus rough value is ok here.
                add_adsorbate(mol_on_slab, mol, height=3, position=(i, j))
                c = FixAtoms(indices=[atom.index for atom in mol_on_slab if atom.position[2] <= z_threshold + 9])
                mol_on_slab.set_constraint(c)

                # Set up the calculator
                mol_on_slab.set_calculator(calc)
                os.makedirs('data', exist_ok=True)
                # Define structure optimizer - LBFGS. Run for 100 steps, 
                # or if the max force on all atoms (fmax) is below 0 ev/A.
                # fmax is typically set to 0.01-0.05 eV/A, 
                # for this demo however we run for the full 100 steps.
                dyn = LBFGS(mol_on_slab, trajectory="data/" + name + "_mol_" + str(mol) + "_onto_" + str(i) + "_" + str(j) + "_relax.traj")#, logfile = None)
                dyn.run(fmax=0.05, steps=300)    
                traj = ase.io.read("data/" + name + "_mol_" + str(mol) + "_onto_" + str(i) + "_" + str(j) + "_relax.traj", ":")
                # convert traj format to extxyz format (used by OC20 dataset)
                columns = (['symbols','positions', 'move_mask', 'tags'])
                with open("data/" + name + "_mol_" + str(mol) + "_onto_" + str(i) + "_" + str(j) + "_relax.extxyz",'w') as f:
                    extxyz.write_xyz(f, traj, columns=columns)

                #最適化後の金属構造をロードし、エネルギーを出力
                raw_slab_energy = slab_energy
                
                

                #最適化後の吸着構造をロードし、エネルギーを出力
                final_structure = traj[-1]
                relaxed_energy = final_structure.get_potential_energy()

                #分子のエネルギー計算（これは関数化すべき）
                adsorbate = Atoms(mol).get_chemical_symbols()
                # For clarity, we define arbitrary gas reference energies here.
                # A more detailed discussion of these calculations can be found in the corresponding paper's SI. 
                gas_reference_energies = {'H': .3, 'O': .45, 'C': .35, 'N': .50}
                adsorbate_reference_energy = 0
                for ads in adsorbate:
                    adsorbate_reference_energy += gas_reference_energies[ads]
                adsorption_energy = relaxed_energy - raw_slab_energy - adsorbate_reference_energy
                print("catalyst:", name, "adsorbate", mol, "site", i, j, "Eads:",  adsorption_energy)
                
                adsorption_structures = adsorption_structures.append({"composition": name,
                                                                      'opt_energy': raw_slab_energy, 
                                                                      "mol": str(mol),
                                                                      'onto_0': i,
                                                                      "onto_1": j,
                                                                      "ads_energy": adsorption_energy}, ignore_index = True)

                #初期構造の表示
                fig, ax = plt.subplots(1, 1)
                ase.visualize.plot.plot_atoms(traj[len(traj) - 1], 
                                              ax, 
                                              radii=0.8, 
                                              rotation=("280x, 60y, 10z"))
                path = "D:/OneDrive/デスクトップ/Data science/ようつべ/実戦編/OCM catalyst reaction analysis" + "/" +  name + str(mol) + str(i) + str(j) + ".png"
                fig.savefig(path)
                plt.show()
    del bulk, slab
    

In [ ]:
adsorption_structures.to_csv("20230107_adsorption_structures_including_CH4_CH4O_O_subsurface_const_bulk_relaxed.csv")

In [ ]:
for index in range(len(df_catalyst_key_unique)):
    ids = df_catalyst_key_unique["Material ID"][df_catalyst_key_unique.index[index]]
    insert_num = df_catalyst_key_unique["M1_atom_number"][df_catalyst_key_unique.index[index]]
    base_num = df_catalyst_key_unique["M2_atom_number"][df_catalyst_key_unique.index[index]]
    insert_ratio = df_catalyst_key_unique["M1_mol%"][df_catalyst_key_unique.index[index]]
    base_ratio = df_catalyst_key_unique["M2_mol%"][df_catalyst_key_unique.index[index]]
    name = df_catalyst_key_unique["Name"][df_catalyst_key_unique.index[index]].replace("/","_")
    print(ids, insert_num, base_num, insert_ratio, name)

    for i in range(1, 4, 1):
        for j in range(1, 3, 1):
            for mol in molec:
                mol = molecule(mol)        
                print(mol, i, j)



In [ ]:
#繰り返し構造の設定
index=0
ids = df_catalyst_key_unique["Material ID"][df_catalyst_key_unique.index[index]]
insert_num = df_catalyst_key_unique["M1_atom_number"][df_catalyst_key_unique.index[index]]
base_num = df_catalyst_key_unique["M2_atom_number"][df_catalyst_key_unique.index[index]]
insert_ratio = df_catalyst_key_unique["M1_mol%"][df_catalyst_key_unique.index[index]]
base_ratio = df_catalyst_key_unique["M2_mol%"][df_catalyst_key_unique.index[index]]
name = df_catalyst_key_unique["Name"][df_catalyst_key_unique.index[index]].replace("/","_")


make_crystal_cif(ids)
bulk = bulk_set_opt(insert_num, base_num, insert_ratio, base_ratio)

slab = makesurface(bulk, miller_indices=(1, 1, 1), layers=2, rep=[1, 1, 1])
slab = sort(slab)
# adjust `positions` before `wrap`
slab.positions += [1, 1, 0]
slab.wrap()

# Check z_position of atoms
atoms = slab
df = pd.DataFrame({
    "x": atoms.positions[:, 0],
    "y": atoms.positions[:, 1],
    "z": atoms.positions[:, 2],
    "symbol": atoms.symbols,
})
display(df)

coord = "z"
df_sorted = df.sort_values(coord).reset_index().rename({"index": "atom_index"}, axis=1)
z_threshold = df_sorted["z"][len(df_sorted) - 20]
fig = px.scatter(df_sorted, x=df_sorted.index, y=coord, color="symbol", hover_data=["x", "y", "z", "atom_index"])
fig.show()

view(atoms)


## 2. 表面の切り出し

結晶構造から表面を切り出します。

In [ ]:
# Fix atoms under z=10 A（subsurfaceの原子は固定、この束縛は任意に緩和可能だが緩和するほど計算は重くなる）
c = FixAtoms(indices=[atom.index for atom in slab if atom.position[2] <= z_threshold])
slab.set_constraint(c)
slab.center(vacuum=10.0, axis=2)
slab.set_pbc(True)
view(slab)

In [ ]:
mol_on_slab = slab.copy()

# height: height of molecule from slab surface
# position: x,y position of molecule
# The molecule position can be modified later, and thus rough value is ok here.
add_adsorbate(mol_on_slab, methoxy, height=3, position=(1, 1))
c = FixAtoms(indices=[atom.index for atom in mol_on_slab if atom.position[2] <= z_threshold + 9])
mol_on_slab.set_constraint(c)
view(mol_on_slab)


In [ ]:
slab = makesurface(bulk, miller_indices=(1, 1, 1), layers=2, rep=[1, 1, 1])
slab = sort(slab)
# adjust `positions` before `wrap`
slab.positions += [1, 1, 0]
slab.wrap()

In [ ]:

# Check z_position of atoms
atoms = slab
df = pd.DataFrame({
    "x": atoms.positions[:, 0],
    "y": atoms.positions[:, 1],
    "z": atoms.positions[:, 2],
    "symbol": atoms.symbols,
})
#display(df)

coord = "z"
df_sorted = df.sort_values(coord).reset_index().rename({"index": "atom_index"}, axis=1)
z_threshold = df_sorted["z"][len(df_sorted) - 50]
fig = px.scatter(df_sorted, x=df_sorted.index, y=coord, color="symbol", hover_data=["x", "y", "z", "atom_index"])
fig.show()

## 3. 組成の変更と構造最適化

先述のとおり今回引用した結晶構造においてはNaが含まれません。ですので、吸着エネルギーの計算にあたってはMnの一部がNaに置換された構造を生成する必要がある、と考えました（実際のところどういう結晶構造なのかというのは作業仮説になってしまうので、ココには論理ギャップがあることに注意する必要があります）。結晶構造中のMnをNaに置換することは非常に簡単ですので、やってみましょう。

In [ ]:
#結晶構造の元素番号を確認する
numbers = atoms.get_atomic_numbers()
numbers

In [ ]:
#Mnの個数
Mn_count = np.count_nonzero(numbers == 25)
#Mnのindex
indices_Mn_count = np.where(numbers == 25)[0]

In [ ]:
#Mnのindexを、任意比率でNaに変更
replace_index = np.random.choice(indices_Mn_count, size= 20, replace=False)
print(f"Replace {replace_index} atom")
numbers[replace_index] = 11
atoms.set_atomic_numbers(numbers)

In [ ]:
# Fix atoms under z=10 A（subsurfaceの原子は固定、この束縛は任意に緩和可能だが緩和するほど計算は重くなる）
c = FixAtoms(indices=[atom.index for atom in slab if atom.position[2] <= z_threshold])
slab.set_constraint(c)
slab.center(vacuum=13.0, axis=2)
slab.set_pbc(True)
slab.set_calculator(calc)

In [ ]:
#構造の確認
fig, ax = plt.subplots(1, 1)
ase.visualize.plot.plot_atoms(slab, 
                              ax, 
                              radii=0.8, 
                              rotation=("-75x, 45y, 10z"))

In [ ]:

os.makedirs("output", exist_ok=True)
BFGS_opt = FIRE(slab, trajectory="output/slab_opt.traj")#, logfile=None)
BFGS_opt.run(fmax=0.01, steps = 500)

In [ ]:
# Save slab structure
os.makedirs("output/structures/", exist_ok=True)
write("output/structures/Slab_MnNa2WO4.xyz", slab)


In [ ]:
#スラブモデルのエネルギーを保存
slab_energy = slab.get_potential_energy()

## 4. 吸着エネルギーの計算



In [ ]:
#吸着種の指定
molec = molecule("O2")
# example to read sdf file
# molec = read("xxxxxx.sdf")

In [ ]:
mol_on_slab = slab.copy()

# height: height of molecule from slab surface
# position: x,y position of molecule
# The molecule position can be modified later, and thus rough value is ok here.
add_adsorbate(mol_on_slab, molec, height=1, position=(8, 4))
c = FixAtoms(indices=[atom.index for atom in mol_on_slab if atom.position[2] <= 8])
mol_on_slab.set_constraint(c)

In [ ]:
#初期構造の表示
fig, ax = plt.subplots(1, 1)
ase.visualize.plot.plot_atoms(mol_on_slab, 
                              ax, 
                              radii=0.8, 
                              rotation=("-75x, 45y, 10z"))

いよいよ吸着エネルギーの計算に移ります。吸着エネルギーは以下の式で定義されます。

Eads = Erelaxed - Eraw_slab - Eadsorbate

つまり、分子を吸着させた系全体のエネルギーが吸着前の構造に比してどれだけ変化したかを指します。ここで、吸着分子のエネルギーについては下記スクリプトをみればわかる通り、非常にラフな計算しかしていません。これは、吸着エネルギーの計算においてEadsobateは定数項としての役割しか果たさず、従って結晶構造を変えた際における吸着エネルギーの相対値に対しては寄与しないためです。

In [ ]:
# Set up the calculator
mol_on_slab.set_calculator(calc)
os.makedirs('data', exist_ok=True)
# Define structure optimizer - LBFGS. Run for 100 steps, 
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A, 
# for this demo however we run for the full 100 steps.
dyn = BFGS(mol_on_slab, trajectory="data/" + "mol_on_slab" + "_relax.traj")
dyn.run(fmax=0.05, steps=1000)    
traj = ase.io.read("data/" + "mol_on_slab" + "_relax.traj", ":")
# convert traj format to extxyz format (used by OC20 dataset)
columns = (['symbols','positions', 'move_mask', 'tags'])
with open("data/" + "mol_on_slab" + "_relax.extxyz",'w') as f:
    extxyz.write_xyz(f, traj, columns=columns)

#最適化後の金属構造をロードし、エネルギーを出力
raw_slab_energy = slab_energy

#最適化後の吸着構造をロードし、エネルギーを出力
final_structure = traj[-1]
relaxed_energy = final_structure.get_potential_energy()

#分子のエネルギー計算（これは関数化すべき）
adsorbate = Atoms(molec).get_chemical_symbols()
# For clarity, we define arbitrary gas reference energies here.
# A more detailed discussion of these calculations can be found in the corresponding paper's SI. 
gas_reference_energies = {'H': .3, 'O': .45, 'C': .35, 'N': .50}
adsorbate_reference_energy = 0
for ads in adsorbate:
    adsorbate_reference_energy += gas_reference_energies[ads]
adsorption_energy = relaxed_energy - raw_slab_energy - adsorbate_reference_energy


最後に吸着エネルギーを確認します。OC20+22力場はOC20力場に比してエネルギーの絶対値が大きくなることが確認されております。　

（簡単な例で確認したところ、相対値は合っていました）。

In [ ]:
#吸着エネルギーの確認
adsorption_energy

In [ ]:
len(traj)

In [ ]:
#初期構造の表示
fig, ax = plt.subplots(1, 1)
ase.visualize.plot.plot_atoms(traj[len(traj) - 1], 
                              ax, 
                              radii=0.8, 
                              rotation=("-75x, 45y, 10z"))
plt.show()

In [ ]:
for i in range(1, 4, 1):
    for j in range(1, 4, 1):
        
        mol_on_slab = slab.copy()

        # height: height of molecule from slab surface
        # position: x,y position of molecule
        # The molecule position can be modified later, and thus rough value is ok here.
        add_adsorbate(mol_on_slab, molec, height=3, position=(i, j))
        c = FixAtoms(indices=[atom.index for atom in mol_on_slab if atom.position[2] <= 8])
        mol_on_slab.set_constraint(c)

        # Set up the calculator
        mol_on_slab.set_calculator(calc)
        os.makedirs('data', exist_ok=True)
        # Define structure optimizer - LBFGS. Run for 100 steps, 
        # or if the max force on all atoms (fmax) is below 0 ev/A.
        # fmax is typically set to 0.01-0.05 eV/A, 
        # for this demo however we run for the full 100 steps.
        dyn = BFGS(mol_on_slab, trajectory="data/" + "mol_on_slab" + "_relax.traj", logfile = None)
        dyn.run(fmax=0.05, steps=1000)    
        traj = ase.io.read("data/" + "mol_on_slab" + "_relax.traj", ":")
        # convert traj format to extxyz format (used by OC20 dataset)
        columns = (['symbols','positions', 'move_mask', 'tags'])
        with open("data/" + "mol_on_slab" + "_relax.extxyz",'w') as f:
            extxyz.write_xyz(f, traj, columns=columns)

        #最適化後の金属構造をロードし、エネルギーを出力
        raw_slab_energy = slab_energy

        #最適化後の吸着構造をロードし、エネルギーを出力
        final_structure = traj[-1]
        relaxed_energy = final_structure.get_potential_energy()

        #分子のエネルギー計算（これは関数化すべき）
        adsorbate = Atoms(molec).get_chemical_symbols()
        # For clarity, we define arbitrary gas reference energies here.
        # A more detailed discussion of these calculations can be found in the corresponding paper's SI. 
        gas_reference_energies = {'H': .3, 'O': .45, 'C': .35, 'N': .50}
        adsorbate_reference_energy = 0
        for ads in adsorbate:
            adsorbate_reference_energy += gas_reference_energies[ads]
        adsorption_energy = relaxed_energy - raw_slab_energy - adsorbate_reference_energy
        print(i, j, adsorption_energy)

        #初期構造の表示
        fig, ax = plt.subplots(1, 1)
        ase.visualize.plot.plot_atoms(traj[len(traj) - 1], 
                                      ax, 
                                      radii=0.8, 
                                      rotation=("-75x, 45y, 10z"))
        plt.show()
